In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 📌 1. 데이터 로드
df = pd.read_csv("jeju_real_estate.csv", encoding='utf-8-sig')

# 📌 2. 평당가격 계산
df['평당가격'] = df['거래금액(만원)'] / (df['전용면적(㎡)'] / 3.3)

# 📌 3. 읍면동 → 행정동 매핑
legal_to_admin = {
    # 일도동
    '일도일동': '일도1동',
    '일도이동': '일도2동',
    '일도1동': '일도1동',
    '일도2동': '일도2동',

    # 이도동
    '이도일동': '이도1동',
    '이도이동': '이도2동',
    '이도1동': '이도1동',
    '이도2동': '이도2동',
    '도남동': '이도2동',

    # 삼도동
    '삼도일동': '삼도1동',
    '삼도이동': '삼도2동',
    '삼도1동': '삼도1동',
    '삼도2동': '삼도2동',

    # 삼양동
    '삼양일동': '삼양동',
    '삼양이동': '삼양동',
    '삼양삼동': '삼양동',
    '삼양1동': '삼양동',
    '삼양2동': '삼양동',
    '삼양3동': '삼양동',
    '도련동': '삼양동',
    '도련일동': '삼양동',
    '도련이동': '삼양동',

    # 용담동
    '용담일동': '용담1동',
    '용담이동': '용담2동',
    '용담삼동': '용담2동',
    '용담1동': '용담1동',
    '용담2동': '용담2동',
    '용담3동': '용담2동',

    # 오라동
    '오라일동': '오라동',
    '오라이동': '오라동',
    '오라삼동': '오라동',
    '오라1동': '오라동',
    '오라2동': '오라동',
    '오라3동': '오라동',

    # 아라동
    '아라일동': '아라동',
    '아라이동': '아라동',
    '아라1동': '아라동',
    '아라2동': '아라동',
    '영평동': '아라동',
    '월평동': '아라동',
    '오등동': '아라동',

    # 외도동
    '외도일동': '외도동',
    '외도이동': '외도동',
    '외도1동': '외도동',
    '외도2동': '외도동',
    '도평동': '외도동',
    '내도동': '외도동',

    # 이호동
    '이호일동': '이호동',
    '이호이동': '이호동',
    '이호1동': '이호동',
    '이호2동': '이호동',

    # 도두동
    '도두일동': '도두동',
    '도두이동': '도두동',
    '도두1동': '도두동',
    '도두2동': '도두동',
    
    # 대천동
    '강정동': '대천동',
    '도순동': '대천동',
    '영남동': '대천동',
    
    # 화북동
    '화북일동': '화북동',
    '화북이동': '화북동',
    
    # 구좌읍
    '한동': '구좌읍',
    '구좌읍': '구좌읍',
    
    # 봉개동
    '회천동': '봉개동',
    '봉개동': '봉개동',
    '용강동': '봉개동',
    
    # 중문동
    '대포동': '중문동',
    '중문동': '중문동',
    '회수동': '중문동',
    '하원동': '중문동',
    
    # 효돈동
    '신효동': '효돈동',
    '하효동': '효돈동',
    
    # 영천동
    '상효동': '영천동',
    '토평동': '영천동',
    
    # 예래동
    '색달동': '예래동',
    '하예동': '예래동',
    '상예동': '예래동',
    
    # 노형동
    '해안동': '노형동',
    '노형동': '노형동',
    
    # 대륜동
    '법환동': '대륜동',
    '서호동': '대륜동', 
    '호근동': '대륜동',
    
    '건입동': '건입동',
    '천지동': '천지동',
    '서귀동': '정방동',
    '보목동': '송산동',
    '서홍동': '서홍동',
    '동홍동': '동홍동',
    '연동': '연동',
    '안덕면': '안덕면',
    '남원읍': '남원읍',
    '표선면': '표선면',
    '한림읍': '한림읍',
    '성산읍': '성산읍',
    '한경면': '한경면',
    '애월읍': '애월읍',
    '추자면': '추자면',
    '대정읍': '대정읍',
    '우도면': '우도면',
    '조천읍': '조천읍'
}
df['읍면동'] = df['시군구'].str.extract(r'([가-힣]+[읍면동])')[0]
df['행정동'] = df['읍면동'].map(legal_to_admin)

# 📌 4. 결측치 제거
df = df.dropna(subset=['행정동', '평당가격', '전용면적(㎡)', '년', '월'])

# 📌 5. 독립변수: 행정동 + 년 + 월 + 전용면적(㎡) 모두 포함
X = df[['행정동', '년', '월', '전용면적(㎡)']]
X = pd.get_dummies(X, columns=['행정동'], drop_first=True)

# 📌 6. 종속변수: 평당가격
y = df['평당가격']

# 📌 7. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 8. 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 📌 9. 예측 및 평가
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"✅ RMSE: {rmse:.2f} 만원")
print(f"✅ R²: {r2:.4f}")

# 📌 10. 변수 중요도 확인
importance = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\n📊 가장 중요한 변수 Top 5:")
print(importance.head(5))

# 📌 11. 미래 예측 (예: 중문동, 2026년 6월, 면적 60㎡ 가정)
target_dong = '행정동_중문동'  # 👉 원하는 동으로 변경

future_data = {col: 0 for col in X.columns}
future_data['년'] = 2026
future_data['월'] = 6
future_data['전용면적(㎡)'] = 60
if target_dong in X.columns:
    future_data[target_dong] = 1

future_df = pd.DataFrame([future_data])

future_pred = model.predict(future_df)
print(f"\n🔮 2026년 6월 중문동(60㎡) 예상 평당가격: {future_pred[0]:,.0f} 만원")


✅ RMSE: 281.62 만원
✅ R²: 0.8033

📊 가장 중요한 변수 Top 5:
전용면적(㎡)     0.711357
년           0.068653
월           0.051376
행정동_노형동     0.035149
행정동_이도2동    0.017064
dtype: float64

🔮 2026년 6월 중문동(60㎡) 예상 평당가격: 1,264 만원
